# Análise Exploratória - Composição da Câmara dos Deputados

Neste notebook será efetuada uma análise exploratória utilizando dados abertos disponibilizados pela Câmara dos Deputados.

A Câmara dos Deputados disponibiliza a lista de deputados via API. Veja a documentação da [API](
http://www2.camara.leg.br/transparencia/dados-abertos/dados-abertos-legislativo/webservices) .

In [1]:
#Importação das bibliotecas a serem utilizadas
import requests
import xml.etree.ElementTree as ET
import datetime
import pandas as pd

In [2]:
# Teste de chamada à API para entender o funcionamento
url = 'http://www.camara.gov.br/SitCamaraWS/Deputados.asmx/ObterDeputados'
r = requests.get(url)
raiz = ET.fromstring(r.text)
deputado = raiz[369]
ET.tostring(deputado)

b'<deputado>\n    <ideCadastro>204371</ideCadastro>\n    <codOrcamento />\n    <condicao>Titular</condicao>\n    <matricula>278</matricula>\n    <idParlamentar>1637543</idParlamentar>\n    <nome>FELIPE RIGONI LOPES</nome>\n    <nomeParlamentar>Felipe Rigoni </nomeParlamentar>\n    <urlFoto>http://www.camara.gov.br/internet/deputado/bandep/204371.jpg</urlFoto>\n    <sexo>masculino</sexo>\n    <uf>ES</uf>\n    <partido>PSB</partido>\n    <gabinete>846</gabinete>\n    <anexo>4</anexo>\n    <fone>3215-5846</fone>\n    <email>dep.feliperigoni@camara.leg.br</email>\n    <comissoes>\n      <titular />\n      <suplente />\n    </comissoes>\n  </deputado>\n  '

In [3]:
# Teste de chamada para obter os detalhes
url_detalhes = 'http://www.camara.leg.br/SitCamaraWS/Deputados.asmx/ObterDetalhesDeputado?ideCadastro={}&numLegislatura=56'
url_detalhes = url_detalhes.format('204527')
r = requests.get(url_detalhes)
raiz_ = ET.fromstring(r.text)
detalhe_deputado = raiz_[0]
ET.tostring(detalhe_deputado)

b'<Deputado>\n    <numLegislatura>56</numLegislatura>\n    <email>dep.geninhozuliani@camara.leg.br</email>\n    <nomeProfissao>\n    </nomeProfissao>\n    <dataNascimento>13/01/1976</dataNascimento>\n    <dataFalecimento>\n    </dataFalecimento>\n    <ufRepresentacaoAtual>SP</ufRepresentacaoAtual>\n    <situacaoNaLegislaturaAtual>Em Exerc&#237;cio</situacaoNaLegislaturaAtual>\n    <ideCadastro>204527</ideCadastro>\n    <idParlamentarDeprecated>699056</idParlamentarDeprecated>\n    <nomeParlamentarAtual>Geninho Zuliani </nomeParlamentarAtual>\n    <nomeCivil>EUGENIO JOS&#201; ZULIANI</nomeCivil>\n    <sexo>M</sexo>\n    <partidoAtual>\n      <idPartido>DEM</idPartido>\n      <sigla>DEM</sigla>\n      <nome>Democratas</nome>\n    </partidoAtual>\n    <gabinete>\n      <numero>860</numero>\n      <anexo>4</anexo>\n      <telefone>3215-5860</telefone>\n    </gabinete>\n    <comissoes>\n      <comissao>\n        <idOrgaoLegislativoCD>538548</idOrgaoLegislativoCD>\n        <siglaComissao>PEC

In [4]:
#Dicionário que será utilizado para pegar os dados interessantes
registro = {
    'ideCadastro': deputado.find('ideCadastro').text,
    'condicao': deputado.find('condicao').text,
    'nome': deputado.find('nome').text,
    'nomeParlamentar': deputado.find('nomeParlamentar').text,
    'sexo': deputado.find('sexo').text,
    'dataNascimento': detalhe_deputado.find('dataNascimento').text,
    'profissao':detalhe_deputado.find('nomeProfissao').text,
    'uf': deputado.find('uf').text,
    'partido': deputado.find('partido').text
}

In [5]:
registro

{'ideCadastro': '204371',
 'condicao': 'Titular',
 'nome': 'FELIPE RIGONI LOPES',
 'nomeParlamentar': 'Felipe Rigoni ',
 'sexo': 'masculino',
 'dataNascimento': '13/01/1976',
 'profissao': '\n    ',
 'uf': 'ES',
 'partido': 'PSB'}

In [6]:
#Obtenção dos conjunto total de dados de forma programática
lista = []
i = 0
for deputado in raiz:
    url_detalhes = 'http://www.camara.leg.br/SitCamaraWS/Deputados.asmx/ObterDetalhesDeputado?ideCadastro={}&numLegislatura=56'
    url_detalhes = url_detalhes.format(deputado.find('ideCadastro').text)
    r = requests.get(url_detalhes)
    raiz_ = ET.fromstring(r.text)
    detalhe = raiz_[0]
    registro = {
    'ideCadastro': deputado.find('ideCadastro').text,
    'condicao': deputado.find('condicao').text,
    'nome': deputado.find('nome').text,
    'nomeParlamentar': deputado.find('nomeParlamentar').text,
    'sexo': deputado.find('sexo').text,  
    'uf': deputado.find('uf').text,
    'partido': deputado.find('partido').text,
    'dataNascimento': detalhe.find('dataNascimento').text,
    'email': deputado.find('email').text
}
    lista.append(registro)
    print(100*i/513)
    i=i+1
df_deputados = pd.DataFrame(lista)

0.0
0.1949317738791423
0.3898635477582846
0.5847953216374269
0.7797270955165692
0.9746588693957114
1.1695906432748537
1.364522417153996
1.5594541910331383
1.7543859649122806
1.949317738791423
2.1442495126705654
2.3391812865497075
2.53411306042885
2.729044834307992
2.9239766081871346
3.1189083820662766
3.313840155945419
3.508771929824561
3.7037037037037037
3.898635477582846
4.093567251461988
4.288499025341131
4.483430799220273
4.678362573099415
4.8732943469785575
5.0682261208577
5.2631578947368425
5.458089668615984
5.653021442495127
5.847953216374269
6.042884990253412
6.237816764132553
6.432748538011696
6.627680311890838
6.822612085769981
7.017543859649122
7.212475633528265
7.407407407407407
7.60233918128655
7.797270955165692
7.992202729044834
8.187134502923977
8.382066276803119
8.576998050682262
8.771929824561404
8.966861598440547
9.161793372319687
9.35672514619883
9.551656920077972
9.746588693957115
9.941520467836257
10.1364522417154
10.331384015594542
10.526315789473685
10.7212475633

87.91423001949317
88.10916179337232
88.30409356725146
88.4990253411306
88.69395711500975
88.88888888888889
89.08382066276803
89.27875243664717
89.47368421052632
89.66861598440546
89.8635477582846
90.05847953216374
90.25341130604288
90.44834307992203
90.64327485380117
90.83820662768031
91.03313840155946
91.2280701754386
91.42300194931774
91.61793372319688
91.81286549707602
92.00779727095517
92.20272904483431
92.39766081871345
92.5925925925926
92.78752436647173
92.98245614035088
93.17738791423002
93.37231968810916
93.5672514619883
93.76218323586744
93.95711500974659
94.15204678362574
94.34697855750487
94.54191033138402
94.73684210526316
94.9317738791423
95.12670565302145
95.32163742690058
95.51656920077973
95.71150097465888
95.90643274853801
96.10136452241716
96.29629629629629
96.49122807017544
96.68615984405459
96.88109161793372
97.07602339181287
97.270955165692
97.46588693957115
97.6608187134503
97.85575048732943
98.05068226120858
98.24561403508773
98.44054580896686
98.63547758284601
9

In [7]:
df_deputados.head()

,condicao,dataNascimento,email,ideCadastro,nome,nomeParlamentar,partido,sexo,uf
0,Titular,26/04/1942,dep.beneditadasilva@camara.leg.br,73701,BENEDITA SOUZA DA SILVA SAMPAIO,Benedita da Silva,PT,feminino,RJ
1,Titular,20/12/1953,dep.angelaamin@camara.leg.br,73696,ANGELA REGINA HEINZEN AMIN HELOU,Angela Amin,PP,feminino,SC
2,Titular,20/04/1959,dep.renildocalheiros@camara.leg.br,73801,RENILDO VASCONCELOS CALHEIROS,Renildo Calheiros,PCdoB,masculino,PE
3,Titular,17/05/1957,dep.jandirafeghali@camara.leg.br,74848,JANDIRA FEGHALI,Jandira Feghali,PCdoB,feminino,RJ
4,Titular,07/04/1947,dep.atilalira@camara.leg.br,74459,ÁTILA FREITAS LIRA,Átila Lira,PP,masculino,PI


In [8]:
#Salvar para consultas futuras e evitar ter que baixar novamente
df_deputados.to_csv('deputados.csv', encoding='latin1', sep=';', index='False')

In [9]:
df_deputados.dtypes

condicao           object
dataNascimento     object
email              object
ideCadastro        object
nome               object
nomeParlamentar    object
partido            object
sexo               object
uf                 object
dtype: object

In [10]:
#Para facilitar a análise, criar uma nova coluna com a Idade calculada utilizando a data de nascomento
# Será utilizada uma função que faz o cáculo
def calcular_idade(dt_nascimento):
    hoje = datetime.date.today()
    return hoje.year - dt_nascimento.year - ((hoje.month, hoje.day) < (dt_nascimento.month, dt_nascimento.day))

In [12]:
df_deputados['idade'] = df_deputados.apply(lambda row: calcular_idade(datetime.datetime.strptime(row.dataNascimento, "%d/%m/%Y")), axis=1)

In [13]:
df_deputados.head()

,condicao,dataNascimento,email,ideCadastro,nome,nomeParlamentar,partido,sexo,uf,idade
0,Titular,26/04/1942,dep.beneditadasilva@camara.leg.br,73701,BENEDITA SOUZA DA SILVA SAMPAIO,Benedita da Silva,PT,feminino,RJ,77
1,Titular,20/12/1953,dep.angelaamin@camara.leg.br,73696,ANGELA REGINA HEINZEN AMIN HELOU,Angela Amin,PP,feminino,SC,66
2,Titular,20/04/1959,dep.renildocalheiros@camara.leg.br,73801,RENILDO VASCONCELOS CALHEIROS,Renildo Calheiros,PCdoB,masculino,PE,60
3,Titular,17/05/1957,dep.jandirafeghali@camara.leg.br,74848,JANDIRA FEGHALI,Jandira Feghali,PCdoB,feminino,RJ,62
4,Titular,07/04/1947,dep.atilalira@camara.leg.br,74459,ÁTILA FREITAS LIRA,Átila Lira,PP,masculino,PI,72


Com os dados estabelecidos, podemos tentar algumas consultas para conhecer melhor o conjunto:

In [14]:
#Quais deputados pertencem ao partido 'CIDADANIA'?
df_deputados[df_deputados['partido'] == 'CIDADANIA']

,condicao,dataNascimento,email,ideCadastro,nome,nomeParlamentar,partido,sexo,uf,idade
5,Titular,23/05/1948,dep.rubensbueno@camara.leg.br,73466,RUBENS BUENO,Rubens Bueno,CIDADANIA,masculino,PR,71
53,Titular,08/05/1955,dep.arnaldojardim@camara.leg.br,141391,ARNALDO CALIL PEREIRA JARDIM,Arnaldo Jardim,CIDADANIA,masculino,SP,64
134,Titular,06/10/1962,dep.carmenzanotto@camara.leg.br,164360,CARMEN EMÍLIA BONFÁ ZANOTTO,Carmen Zanotto,CIDADANIA,feminino,SC,57
147,Titular,07/07/1982,dep.marcelocalero@camara.leg.br,204433,MARCELO CALERO FARIA GARCIA,Marcelo Calero,CIDADANIA,masculino,RJ,37
170,Titular,04/11/1978,dep.danielcoelho@camara.leg.br,178916,DANIEL PIRES COELHO,Daniel Coelho,CIDADANIA,masculino,PE,41
199,Titular,22/08/1979,dep.alexmanente@camara.leg.br,178972,ALEX SPINELLI MANENTE,Alex Manente,CIDADANIA,masculino,SP,40
246,Titular,02/06/1971,dep.davitoria@camara.leg.br,204355,JOSIAS MARIO DA VITORIA,Da Vitoria,CIDADANIA,masculino,ES,48
403,Titular,23/06/1973,dep.paulabelmonte@camara.leg.br,204377,PAULA MORENO PARO BELMONTE,Paula Belmonte,CIDADANIA,feminino,DF,46


In [15]:
# Qual a distribuição entre os partidos dos deputados de SP
df_deputados[df_deputados['uf'] == 'SP'].partido.value_counts()

PSL              9
PSDB             9
PT               8
PL               7
REPUBLICANOS     6
DEM              4
PP               4
PODE             3
PSOL             3
PSB              3
NOVO             3
MDB              2
CIDADANIA        2
PSD              2
PDT              1
PSC              1
PCdoB            1
SOLIDARIEDADE    1
PV               1
Name: partido, dtype: int64

In [ ]:
#Distribuição dos deputados entre os partidos
#

In [ ]:
# Distribuição dos deputados por sexo
#

Que outras perguntas poderiam ser feitas?? Imagine que você trabalha para uma revista e precisa fazer um infográfico sobre a composição da Câmara na nova legistatura.